In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

from data_generator import DataGenerator

import model as model_util


import matplotlib.pyplot as plt

from numpy import array
import numpy as np


import datetime
from datetime import datetime

from tensorflow.keras.layers import BatchNormalization, GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences


from tensorflow.keras import datasets, layers, models

import tensorflow as tf

#%load_ext tensorboard
print(tf.__version__)

2.1.0


In [2]:
def reset_seed():
    seed_value= 0
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    import random
    random.seed(seed_value)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    import numpy as np
    np.random.seed(seed_value)
    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    import tensorflow as tf
    tf.random.set_seed(seed_value)
    
reset_seed()


In [6]:
import random


path = "drive/My Drive/model/"
path = "./"

trainX = np.load(path + "train" + "X.npy")
trainY = np.load(path + "train" + "Y.npy")
print("Train")
print(trainX.shape)
print(trainY.shape)

valX = np.load(path + "val" + "X.npy")
valY = np.load(path + "val" + "Y.npy")
print("Validation")
print(valX.shape)
print(valY.shape)

def get_balanced_set(X, Y):
    positiveX = [X[0]]
    positiveY = [Y[0]]
    negativeX = [X[0]]
    negativeY = [Y[0]]
    from tqdm import tqdm_notebook as tqdm

    for i in tqdm(range(1, len(X)-1)):
        x = X[i]
        y = Y[i]
        if (y == 1):
            positiveX.append(x)
            positiveY.append(y)
        else:
            negativeX.append(x)
            negativeY.append(y)


    positiveX = np.array(positiveX)
    positiveY = np.array(positiveY)
    negativeX = np.array(negativeX)
    negativeY = np.array(negativeY)

    negative_num = len(positiveY)
    start_index = random.randint(0,(len(negativeX) - negative_num))
    end_index = start_index + negative_num

    print(start_index)
    print(end_index)

    trainX = np.concatenate((positiveX, negativeX[start_index:end_index]), axis=0)
    trainY = np.concatenate((positiveY, negativeY[start_index:end_index]), axis=0)
    return trainX, trainY, positiveX, positiveY, negativeX, negativeY


trainX, trainY, positiveX, positiveY, negativeX, negativeY = get_balanced_set(trainX, trainY)

valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = get_balanced_set(valX, valY)

print("Train:")

print("positiveX {0}".format(positiveX.shape))
print("positiveY {0}".format(positiveY.shape))
print("negativeX {0}".format(negativeX.shape))
print("negativeY {0}".format(negativeY.shape))

print("Validation:")

print("positiveX_ {0}".format(positiveX_.shape))
print("positiveY_ {0}".format(positiveY_.shape))
print("negativeX_ {0}".format(negativeX_.shape))
print("negativeY_ {0}".format(negativeY_.shape))

print("Final:")

print(trainX.shape)
print(trainY.shape)
print(valX.shape)
print(valY.shape)

Train
(883497, 16)
(883497,)
Validation
(2658, 16)
(2658,)


/home/samir/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



441001
562740



82
519
Train:
positiveX (121739, 16)
positiveY (121739,)
negativeX (761758, 16)
negativeY (761758,)
Validation:
positiveX_ (437, 16)
positiveY_ (437,)
negativeX_ (2221, 16)
negativeY_ (2221,)
Final:
(243478, 16)
(243478,)
(874, 16)
(874,)


In [12]:
# define generator
n_input = 80

class CustomGen(TimeseriesGenerator):
    def __getitem__(self, idx):
        x,y = super().__getitem__(idx)
        return x, y

def get_gen(set_x, set_y, shuffle=True, batch_size=300):
    return CustomGen(set_x, set_y, length=n_input, batch_size=batch_size, shuffle=shuffle)

train_generator = get_gen(trainX, trainY, shuffle=True)

val_generator = get_gen(valX, valY, shuffle=False)

# number of samples
print('Samples: %d' % len(train_generator))

x, y = train_generator[0]

x_, y_ = val_generator[0]

features = x.shape[2]

print(x.shape)
print(y.shape)

Samples: 812
(300, 80, 16)
(300,)


In [17]:
filepath_encoder = "encoder.h5"

def load_model(filepath = filepath_encoder):
    model_loaded = tf.keras.models.load_model(filepath)
    return model_loaded

def load_encoder():
    model_loaded = load_model(filepath = filepath_encoder)
    encoder = Model(inputs=model_loaded.input, outputs=model_loaded.get_layer('encoder').output)
    print("Number of layers in the base model: ", len(model_loaded.layers))
    print("Number of layers in the encoder: ", len(encoder.layers))
    encoder.trainable = False
    for layer in encoder.layers:
        layer._name = layer.name + str("_encoder")
    return encoder

encoder = load_encoder()

Number of layers in the base model:  57
Number of layers in the encoder:  25


In [18]:
inputX = encoder.output

dim = 20

def get_classifier_model():
    def add_deep_layers(input_layer, units = features):
        x = Dense(units, activation='relu')(input_layer)
        x = Dropout(0.2)(x)
        x = Dense(features, activation='relu')(x)
        return x
    
    x = Dense(dim, name = "input")(inputX)
    x = Dropout(0.2)(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
    x = add_deep_layers(x)
    x = add_deep_layers(x)
    x = add_deep_layers(x)
    x = Dense(100)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
    x = Dense(20)(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.3)(x)
    x = Dense(1, activation='sigmoid')(x)
    return Model(inputs=encoder.input, outputs=x)
    

model = get_classifier_model()



In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001, epsilon=1e-6),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [20]:

filepath = "stock.h5"


checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss',
                             verbose=1,
                             save_best_only=True,
                             load_weights_on_restart=True)

callbacks_list = [checkpoint]

history = model.fit(
    #x, x,
    train_generator,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    steps_per_epoch=len(train_generator), 
    epochs=300, 
    verbose=1, 
    callbacks=callbacks_list)


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 812 steps, validate for 3 steps
Epoch 1/300
811/812 [============================>.] - ETA: 0s - loss: 0.7193 - accuracy: 0.4994
Epoch 00001: loss improved from inf to 0.71932, saving model to stock.h5
812/812 [==============================] - 142s 175ms/step - loss: 0.7193 - accuracy: 0.4993 - val_loss: 0.7094 - val_accuracy: 0.4496
Epoch 2/300
811/812 [============================>.] - ETA: 0s - loss: 0.7091 - accuracy: 0.5006
Epoch 00002: loss improved from 0.71932 to 0.70907, saving model to stock.h5
812/812 [==============================] - 155s 191ms/step - loss: 0.7091 - accuracy: 0.5006 - val_loss: 0.7093 - val_accuracy: 0.4496
Epoch 3/300
811/812 [============================>.] - ETA: 0s - loss: 0.7089 - accuracy: 0.4995
Epoch 00003: loss improved from 0.70907 to 0.70892, saving model to stock.h5
812/812 [==============================] - 147s 181ms/step - loss: 0.7089 - accuracy: 0.4995 - val_loss: 0.7098 - val_ac

KeyboardInterrupt: 

In [ ]:
def show_loss(history): 
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

show_loss(history)

In [ ]:
def show_accuracy(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_loss'])
    plt.title('accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

show_accuracy(history)

In [ ]:
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.losses import mean_squared_logarithmic_error, mean_squared_error

def error(y_true, y_pred):
    return np.mean(mean_squared_logarithmic_error(y_true, y_pred))

def show_result(modell):
    pred = modell.predict(x_)
    score1 = error(pred,x_)
    print(f"Out of Sample Score (RMSE): {score1}")
    pred = modell.predict(x)
    score2 = error(pred,x)
    print(f"Insample Normal Score (RMSE): {score2}")



In [69]:
positive = 0
negative = 0
for i in range(1, len(positiveY_)-1):
    y = positiveY_[i][0]
    if (y == 1):
        positive += 1
    else:
        negative += 1
        
print(positive)
print(negative)

435
0


In [57]:
val_generator_ = get_gen(positiveX_, positiveY_, shuffle=False)
val_generator_ = get_gen(negativeX_, negativeY_, shuffle=False)
#val_generator_ = get_gen(negativeX_, negativeY_, shuffle=False)

In [58]:
x, y = val_generator_[0]

In [68]:
test = np.array([x[1]])
test.shape

(1, 80, 16)

In [76]:
preds = model.predict(x)